In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
import re

##### Step 1: Indexing -- Document Ingestion

In [34]:
id = "SfOaZIGJ_gs"  # Video id from youtube, extract from URL for future works
YTapi = YouTubeTranscriptApi()
try:
    transcript_list = YTapi.fetch(id, languages = ["en"])
    #
    # The API provides results in a FetchecTranscriptSnippet object, in the following format:
    #snippets=[
        # FetchedTranscriptSnippet(
        #     text="Hey there",
        #     start=0.0,
        #     duration=1.54,
        # ),
        # FetchedTranscriptSnippet(
        #     text="how are you",
        #     start=1.54,
        #     duration=4.16,)
    #
    # The resulting transcript will be provided on the basis of timestamps, (when the text starts and till what duration it is maintained.)
    # The API gets the transcript sentence by sentence in a broken format, so we need to join it into a single plain text.
    #
    # Join text including the verbal cues
    raw_transcript = " ".join(chunk.text for chunk in transcript_list)
    #
    # Clean the text by removing these non-verbal cues using regex functions
    clean_transcript = re.sub(r"\[.*?\]", "", raw_transcript)
    clean_transcript = re.sub(r"\s+", " ", clean_transcript).strip()
    print(clean_transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

uh where Nik you told him 5 minutes like he has 2 minutes not yeah 2 minutes Everything looks good. Just the monitor, the main monitor uh went off. Everyone who's done can leave. Hi, fam. Hey, Nicole. How are you? I'm good. Sorry I'm late. I got caught up in getting ready for the launch tomorrow and lost track of time and excitement with the final results. But no worries. I'm guessing it must be really hectic, right? It is a very hectic day. I have the model and I've been playing with it a little bit. How is it different, Sam? I'm not an expert at this. So yeah, there's all these ways we can talk about, you know, it's better at this metric or it's, you know, can do this amazing coding demo that the, you know, GPT4 couldn't. But the thing that has been most striking for me is in ways that are both big and small, going back from GPT5 to our previous generation model is just so painful. It's just like worse at everything. And I've taken for granted that there is a fluency and a depth of i

##### Text Splitting

In [35]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([clean_transcript])

In [36]:
len(chunks)

49

In [37]:
chunks[0]

Document(metadata={}, page_content="uh where Nik you told him 5 minutes like he has 2 minutes not yeah 2 minutes Everything looks good. Just the monitor, the main monitor uh went off. Everyone who's done can leave. Hi, fam. Hey, Nicole. How are you? I'm good. Sorry I'm late. I got caught up in getting ready for the launch tomorrow and lost track of time and excitement with the final results. But no worries. I'm guessing it must be really hectic, right? It is a very hectic day. I have the model and I've been playing with it a little bit. How is it different, Sam? I'm not an expert at this. So yeah, there's all these ways we can talk about, you know, it's better at this metric or it's, you know, can do this amazing coding demo that the, you know, GPT4 couldn't. But the thing that has been most striking for me is in ways that are both big and small, going back from GPT5 to our previous generation model is just so painful. It's just like worse at everything. And I've taken for granted that

##### Embedding Generation and Storage in a Vector Store

In [38]:
#using the following embedding model
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")
#we provide our chunks along with our embedding model to the vector store to create an embedding for each chunk
vector_store = FAISS.from_documents(chunks, embeddings)

In [39]:
#we get an embedding id against all our chunks, which can be used to call an individual chunk
vector_store.index_to_docstore_id

{0: '8d6e8d93-c5cf-48b8-9056-1adb6c31b79c',
 1: '49d81098-6be0-4e4e-8ce6-73d29c9e830d',
 2: '0b5bc92b-1352-44ce-9aef-1e1131dca3ed',
 3: 'f28c1012-93d0-48bc-883c-e171018cc4aa',
 4: '3b260299-c388-40b4-9a21-3fba3708ee82',
 5: '4b2d15a4-2e08-4f57-b7fb-5a98084f2ee8',
 6: '041581ca-1e4a-4ee9-88c8-e00832c12de2',
 7: '0f32a530-0705-4a13-8be5-4e907466934b',
 8: '1678010f-a220-4ec4-a98d-e15104fb12fd',
 9: '958970b2-9e78-4002-bbac-425d5dbbbc29',
 10: '9f90264d-6cb0-42d3-abd7-4f03ab48b1cc',
 11: '7841980d-0a84-4287-8c21-b0d91b7be254',
 12: '537554cb-dc6b-4037-8f7c-b9b7a23d876e',
 13: '527a6775-92df-461c-9afd-e6474fee590b',
 14: '9fe29e88-c24e-400c-b7eb-ea5e59a6556e',
 15: 'fd0885e8-10b4-4bef-88e8-bc1cab79fd92',
 16: '9fd03847-0cf8-42a4-8f09-23f99b89f1e4',
 17: '96582017-ecb8-4cdd-9d42-e70bf0b34738',
 18: '1ef2ae08-2c10-4444-9f2c-d38ef523f55a',
 19: '2985e805-72d5-46c4-a64f-6580f616c160',
 20: '9c570f67-76cb-4b50-9416-fc3116dbfb86',
 21: '0675849c-b693-4bdb-baef-dd6fe831bab7',
 22: '800ad288-bff8-

In [41]:
#vector_store.get_by_ids(['44448d11-e485-47f5-a0f5-0875921efb55'])

##### Step 2: Retrievel

In [42]:
# Forming a retriever using our vector store, that will use a simple similarity search and get 4 most similar results in the output.
#
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 4})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1119b56d0>, search_kwargs={'k': 4})

In [43]:
# Providing a query to the retriever to find the most similar documents. This query is converted into an embedding and then semantic search is performed.
#
retriever.invoke("What is the future of Robotics?")

[Document(id='26898d87-c33d-450e-b827-3e22f29214f4', metadata={}, page_content="a thousand hour problem but certainly in the world today we cannot at all and so that that's like another dimension where AI can't do it. So I was in the US between SF between San Francisco and New York the last couple of months and I met a whole bunch of AI founders. Uh the one thing everybody seemed to agree on is like for AI the US is a few years ahead of most others. They also thought that for robotics China seems to be ahead. Uh do you have a view on robotics and what happens there like humanoid or other form of robotics? I think it will be incredibly important in a couple of years. I think uh one of the things that is going to feel most AGI like is seeing robots just walk by you on the street doing kind of normal day-to-day tasks. Um is there a reason why they need to have humanlike form? Well, you can certainly have non-humanoid forms, but the world is really built for humans, you know, like door han

##### Step 3: Augmentation

In [44]:
# Forming an LLM
#
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0.2)

In [45]:
# Creating a prompt template with user query and retrieved context(documents) to be provided to the LLM
#
prompt = PromptTemplate(
    template = """
        You are a helpful assistant.
        Answer ONLY from the provided transcript context.
        If the context is insufficient, just say you do not know.

        {context}
        Question: {question}
    """,
    input_variables= ['context', 'question']
)

In [46]:
question = "Is the topic of Parenthood discussed in this video? If yes what was discussed?"
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(id='9c570f67-76cb-4b50-9416-fc3116dbfb86', metadata={}, page_content="this than most because of your role at Y Combinator. I have a lot of data points on it at least. Yeah. Yeah. Uh when we met in Washington a couple of days a couple of years ago at the White House, I remember when we were speaking and you went somewhere, I was speaking to your partner and you guys had a kid. We did. And how is that? Uh it is my favorite thing ever. But I mean I I know that like I have nothing that is not a cliche to say here. Um, but it is the coolest, most amazing, most like emotionally overwhelming in the best ways and hard ways to uh experience every everything said about how great it is, how intense it is, how it's like a kind of love you didn't know you could feel. It's all it's all true. I have nothing to add other than I strongly recommend it and I think uh it's been really wonderful. It's amazing. So I ponder on this a lot Sam uh kids, why people have kids and also questions like wha

In [47]:
#creating context from the page_content of retrieved docs to be sent to the LLM for generation
#
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"this than most because of your role at Y Combinator. I have a lot of data points on it at least. Yeah. Yeah. Uh when we met in Washington a couple of days a couple of years ago at the White House, I remember when we were speaking and you went somewhere, I was speaking to your partner and you guys had a kid. We did. And how is that? Uh it is my favorite thing ever. But I mean I I know that like I have nothing that is not a cliche to say here. Um, but it is the coolest, most amazing, most like emotionally overwhelming in the best ways and hard ways to uh experience every everything said about how great it is, how intense it is, how it's like a kind of love you didn't know you could feel. It's all it's all true. I have nothing to add other than I strongly recommend it and I think uh it's been really wonderful. It's amazing. So I ponder on this a lot Sam uh kids, why people have kids and also questions like what happens to religion and marriage tomorrow? Can I ask you why you had a kid?\n

In [48]:
# Combining context and question into the prompt
#
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="\n        You are a helpful assistant.\n        Answer ONLY from the provided transcript context.\n        If the context is insufficient, just say you do not know.\n\n        this than most because of your role at Y Combinator. I have a lot of data points on it at least. Yeah. Yeah. Uh when we met in Washington a couple of days a couple of years ago at the White House, I remember when we were speaking and you went somewhere, I was speaking to your partner and you guys had a kid. We did. And how is that? Uh it is my favorite thing ever. But I mean I I know that like I have nothing that is not a cliche to say here. Um, but it is the coolest, most amazing, most like emotionally overwhelming in the best ways and hard ways to uh experience every everything said about how great it is, how intense it is, how it's like a kind of love you didn't know you could feel. It's all it's all true. I have nothing to add other than I strongly recommend it and I think uh it's been

##### Step 4: Generation

In [49]:
#invoke the LLM with our final prompt and display the result
#
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of parenthood is discussed in the video. The speaker shares their personal experience of having a child, describing it as the "coolest, most amazing" and "emotionally overwhelming" experience. They express that having a family feels like the most important and meaningful thing they could do, and it has exceeded their expectations. The discussion also touches on the societal implications of family and community, suggesting that these aspects will become more important in a future influenced by advancements like AGI (Artificial General Intelligence). The speaker reflects on the value of creating family and community, noting that many people they know who have had both a career and a family regard having children as one of the best things they've done in their lives.


#### Building a Chain

In [50]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [51]:
def format_docs(retrieved_docs):
    context_text = " ".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [ ]:
# We will recieve context and question for our prompt simultaenously, to be provided to the prompt.
# The context and question will be provided to LLM with a prompt for the response.
#
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [53]:
parallel_chain.invoke("who is Sam?")

{'context': "I think the world will experiment with a lot of those. But this idea of sort of ambiently aware physical hardware that feels like it's going to be important. Is that the form factor with Johnny Ive like a sensor? Uh so we'll try multiple products. Uh but but I think this idea of trying to build hardware that an AI companion can sort of embody itself in will be an important thread. Right. The the last two things I want to ask you Sam is one about fusion because I know you've invested in helion and you're a big proponent of it. Does that solve the climate change problem? Would you put money on that? Well, it certainly helps a lot. I I suspect that we've already done sufficient damage to the climate. We're going to have to undo some damage, too, even if we got to switch to fusion right away. But it would certainly be a great step forward. And the last question I have for you Sam is the question I care most about. What's in this AI realm for India as a country? What's the oppo

In [54]:
# Create a parser, as we need to look at the ouput in string format.
#
parser = StrOutputParser()

In [55]:
main_chain = parallel_chain | prompt | llm | parser

In [56]:
main_chain.invoke('Can you Summarize the video?')

'The video features a discussion about the advancements in AI, particularly focusing on the new GPT-5 model compared to previous versions. Sam highlights the significant improvements in fluency and intelligence of GPT-5, describing it as akin to having PhD-level experts available 24/7 for various tasks, from software creation to research reports. The conversation also touches on the entrepreneurial energy in India regarding AI development and the potential for hardware that integrates AI companions. Additionally, Sam discusses his investment in fusion energy, acknowledging its potential to help with climate change but expressing concern about the damage already done. Overall, the discussion emphasizes the transformative impact of AI and the opportunities it presents, especially for India.'

- Video Summarization:
    'The video features a discussion about the advancements in AI, particularly focusing on the new GPT-5 model compared to previous versions. Sam highlights the significant improvements in fluency and intelligence of GPT-5, describing it as akin to having PhD-level experts available 24/7 for various tasks, from software creation to research reports. The conversation also touches on the entrepreneurial energy in India regarding AI development and the potential for hardware that integrates AI companions. Additionally, Sam discusses his investment in fusion energy, acknowledging its potential to help with climate change but expressing concern about the damage already done. Overall, the discussion emphasizes the transformative impact of AI and the opportunities it presents, especially for India.'